# Character segmentation for license plate recognition

Necessary packages

In [2]:
import cv2, os, time
import numpy as np

## Inference for a single image
This function takes an image and returns True or False wheter or not it found a license plate.

In [28]:
def isPlate(img):
    a_Plate = False
    saveChar = True
    contImages2 = 0
    
    kernel = np.ones((2,2),np.uint8)
    mser = cv2.MSER_create()
    # gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    vis = img.copy()
    
    # collect bounding boxes from the image
    regions = mser.detectRegions(gray)
    bboxes = identifyBoundingBoxes(regions,vis)
    clases = calcularClases(bboxes)
    
    if len(clases) == 6:
        a_Plate = True
        for areas2 in clases:
            if saveChar:
                lower_range = np.array([0,0,0])
                upper_range = np.array([180,255,80])
                lower_range2 = np.array([0,0,0])
                upper_range2 = np.array([70,70,70])
                imgSaved = vis[areas2[1]:areas2[1]+areas2[3],areas2[0]:areas2[0]+areas2[2]]
                hsv = cv2.cvtColor(imgSaved, cv2.COLOR_BGR2HSV)
                maskf = cv2.inRange(src=hsv, lowerb=lower_range, upperb=upper_range)
                cv2.rectangle(vis,(areas2[0],areas2[1]),(areas2[0]+areas2[2],areas2[1]+areas2[3]),(0,255,0))
            
                _,thresh2 = cv2.threshold(maskf,127,255,cv2.THRESH_BINARY_INV)
                cv2.imwrite(str(contImages2)+'.png',thresh2)
                contImages2+=1
                
            else:
                cv2.rectangle(vis,(areas2[0],areas2[1]),(areas2[0]+areas2[2],areas2[1]+areas2[3]),(255,255,255))
    if len(clases)==6:
        print('buena')
    return a_Plate

### Identifying bounding boxes
From the regions 'regions' generated in a single image 'img', this returns the accepted bunding boxes for the license plate, according to size and location.

In [29]:
def identifyBoundingBoxes(regions, vis):
    nh = img.shape[1]
    nw = img.shape[0]
    ltmp = list(regions[1])
    sorted_y_idx_list = sorted(range(len(ltmp)),key=lambda x: ltmp[x][0])
    nregions1 = [ltmp[i] for i in sorted_y_idx_list ]
    nregions0 = [regions[0][i] for i in sorted_y_idx_list ]
    
    bboxes = []
    contbbox = 0
    coo = [x[0] for x in nregions1]
    c2 = np.array(coo).reshape(-1,1)

    for p in nregions0:
        ax = cv2.contourArea(p)
        a = cv2.convexHull(p.reshape(-1, 1, 2))
        x, y, w2, h2 = nregions1[contbbox]
        ax = w2*h2
        
        if ax > 0.02*(nh*nw) and ax < 0.16*(nh*nw) and h2>w2 and h2<3*w2:
            cv2.rectangle(vis,(x,y),(x+w2,y+h2),(255,20,25))
            # Calculate the center of the region
            M = cv2.moments(a)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
            else:
                cY = nw

            if (0.25*nw) < cY < (0.65*nw):
                # append the accepted bounding box
                bboxes.append(nregions1[contbbox])
                
        contbbox+=1 
    
    return bboxes    

### Helper function
Calculate the Intersection over Union (IoU) of two bounding boxes. [Calculate iou](https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation)

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]

In [30]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

### Helper function
Calculating the classes for the bounding boxes, in case there is some that are overlapped. This takes as an input the accepted bounding boxes, 'bboxes', and returns a list containing the final bounding boxes.

In [31]:
def calcularClases(bboxes):
    clases = [[]]
    if bboxes == []:
        return []
    for b in bboxes:
        for i in range(len(clases)):
            l = clases[i]
            if l == []:
                l.append(b)
            else:
                ll = l[-1]
                db = {'x1':b[0], 'x2':b[0]+b[2], 'y1':b[1], 'y2':b[1]+b[3]}
                dl = {'x1':ll[0], 'x2':ll[0]+ll[2], 'y1':ll[1], 'y2':ll[1]+ll[3]}
                ii = get_iou( db, dl)
                if ii > 0.05:
                    l.append(b)
                    break
                else:
                    if i == len(clases)-1:
                        clases.append([b])
                        break
    clas = [0]*(len(clases))
    for i in range(len(clases)):
        l = clases[i]
        clas[i] = max(l, key = lambda x: x[1]*x[2])
    
    return clas

## Run inference in the archives
Make sure to link the correct directory!

In [32]:
files = os.listdir('ImagesDetection')
archivos = [cv2.imread(i) for i in files]
for i in range(len(archivos)):
    print(isPlate(archivos[i]))

NameError: name 'vis' is not defined